In [1]:
!conda install -yc conda-forge geocoder lxml cssselect geopy

import urllib.parse
import urllib.request
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import geocoder 
from geopy.geocoders import Nominatim
import time
import xml.etree.ElementTree as ET

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - cssselect
    - geocoder
    - geopy
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    geographiclib-1.50         |           

In [2]:
location = 'munich'

In [3]:
req = urllib.request.Request(
    url = "http://api.glassdoor.com/api/api.htm?v=1&format=json&action=jobs&l={}".format(location),
    headers = {'Host': 'api.glassdoor.com', 'User-Agent': 'curl/7.58.0'}
)
url = urllib.request.urlopen(req)
json_data = json.loads(url.read().decode())
pages = json_data['response']['totalNumberOfPages'] if json_data['response']['totalNumberOfPages'] < 31 else 30

print("Getting {} pages".format(pages))
jobs = []
for page in range(pages):
    print(page + 1, end = " ")
    req = urllib.request.Request(
        url = "http://api.glassdoor.com/api/api.htm?v=1&format=json&action=jobs&l={}&pn={}".format(location, page + 1), 
        headers = {'Host': 'api.glassdoor.com', 'User-Agent': 'curl/7.58.0'}
    )
    url = urllib.request.urlopen(req)
    content = url.read().decode()
    json_data = json.loads(content)
    jobs += json_data['response']['jobListings']

Getting 30 pages
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

In [4]:
df_jobs = json_normalize(jobs)
df_jobs = df_jobs.groupby(['source']).count().iloc[:,0:1]
df_jobs = df_jobs.rename(columns={'jobListingId':'Positions'}).rename_axis('Company').sort_values('Company')
df_jobs.to_csv('{}-jobs.csv'.format(location))

In [5]:
df_jobs = pd.read_csv('{}-jobs.csv'.format(location))

In [6]:
BING_KEY = 'Aq7bYrs0DLI7HUs7AbCd0Ejxsr72-c_RVKm0t1p6ivr0Dne8fbEdie88uM9rQlvS'
def get_location_bing(address):
    time.sleep(1)
    print(address, end = ' => ')
    geo = geocoder.bing(address, key = BING_KEY)
    #print("{}: {}".format(zip, geo.latlng))
    print('.')
    return pd.Series(geo.latlng, index = ('Latitude', 'Longitude'))

geolocator = Nominatim(user_agent="ra_ex")
def get_location(address):
    print(address, end = ' => ')
    attempts = 5
    location = None
    for attempt in range(attempts):
        time.sleep(1)
        try:
            location = geolocator.geocode(address)
            break
        except:
            print(attempt, end = '')
    #print('.', end = '')
    if location:
        print(location)
        return pd.Series((location.latitude, location.longitude), index = ('Latitude', 'Longitude'))
    else:
        print(" X")
        return pd.Series((0, 0), index = ('Latitude', 'Longitude'))
    
def get_location_cz(address):
    print(address, end = ' => ')
    url = urllib.request.urlopen('http://api.mapy.cz/geocode?query={}'.format(urllib.parse.quote(address)))
    resp = url.read().decode()
    doc = ET.fromstring(resp)
    #print(resp)
    if len(doc.getchildren()[0].getchildren()) == 0:
        print("Not found")
        return pd.Series((0, 0), index = ('Latitude', 'Longitude'))
    else:
        entry = doc.getchildren()[0].getchildren()[0]
        print("Found -> {}".format(entry.get('title')))
        result = pd.Series((float(entry.get('y')), float(entry.get('x'))), index = ('Latitude', 'Longitude'))
        #print(result)
        return result

In [7]:
area = '{}, Munich, Germany'
# First we look for geocoding in Nominatim
df_jobs = df_jobs.drop(columns = ['Latitude', 'Longitude'], errors = 'ignore').join(
    df_jobs['Company'].apply(lambda n: get_location(area.format(n))))
# Secondly - in mapy.cz
df_jobs.update(df_jobs[df_jobs['Latitude'] == 0].apply(lambda n: get_location_cz(area.format(n[0])), axis = 1))

ACI Worldwide, Munich, Germany =>  X
APCO Worldwide, Munich, Germany =>  X
ASG Technologies, Munich, Germany =>  X
Abercrombie & Fitch, Munich, Germany => Abercrombie & Fitch, 8, Sendlinger Straße, Hofstatt, Bezirksteil Hackenviertel, Stadtbezirk 01 Altstadt-Lehel, München, Bayern, 80331, Deutschland
Acanthos Ventures GmbH, Munich, Germany =>  X
Adobe, Munich, Germany =>  X
Advantech, Munich, Germany =>  X
Adyen, Munich, Germany =>  X
Ageras, Munich, Germany =>  X
Agoda, Munich, Germany =>  X
Alcon, Munich, Germany =>  X
Alira Health, Munich, Germany =>  X
AlixPartners, Munich, Germany =>  X
Allen & Overy, Munich, Germany =>  X
Allen and Overy, Munich, Germany =>  X
Allianz Deutschland AG, Munich, Germany =>  X
Allianz Global Investors, Munich, Germany => Allianz Global Investors, 24-24a, Seidlstraße, Bezirksteil Augustenstraße, Stadtbezirk 03 Maxvorstadt, München, Bayern, 80335, Deutschland
Allianz Partners, Munich, Germany => Allianz Capital Partners, 24-24a, Seidlstraße, Bezirksteil

In [8]:
df_jobs.to_csv('{}-jobs.csv'.format(location), index = False)

In [9]:
df_jobs[df_jobs['Latitude'] != 0].count()

Company      55
Positions    55
Latitude     55
Longitude    55
dtype: int64

In [10]:
df_jobs.sort_values('Positions', ascending = False)

,Company,Positions,Latitude,Longitude
60,Deutsche Bahn,82,48.126900,11.606434
213,neuvoo,55,0.000000,0.000000
179,Siemens PLC,47,0.000000,0.000000
96,Indeed XML Feed Sponsored Underpacing,44,0.000000,0.000000
90,Hilti Group,43,0.000000,0.000000
...,...,...,...,...
42,CV-Library.co.uk,1,0.000000,0.000000
117,Logicalis GmbH,1,0.000000,0.000000
118,Looker,1,0.000000,0.000000
41,CMS Hasche Sigle,1,48.148131,11.556789
